# Project Summary: SDR Quota Fairness & Bayesian Forecasting

I set out to build a fair and data-driven quota-setting system for Sales Development Representatives (SDRs), and also estimating the probability that each SDR will meet or exceed their sales target using Bayesian modeling.

## Fair Quota Setting by Territory

Analyzed 8 historical quarters of territory-level performance.
Forecasted Q3 2015 quotas per territory using a weighted average:

60% recent quarter (Q2)
40% historical average

Applied a ±35% cap to avoid unfair swings due to outliers or noise.


## SDR-Level Target Allocation

Measured each SDR’s sales contribution to their territory in Q2.
Multiplied that contribution ratio by their territory’s Q3 forecast to create a custom Q3 target per SDR.
→ This ensured targets were relative to territory growth and grounded in recent performance.


## Bayesian Probability of SDR Success

Built a Bayesian model for each SDR–territory pairing using their historical average and standard deviation of sales, excluding Q3.
The model outputs included:

Posterior mean sales
Credible interval (94%)
Probability of hitting their Q3 forecast
Risk flag: high, moderate, or low likelihood of success


## Outcome
This approach ensured:

Fair territory quotas,
Personalized SDR targets based on real contribution,
And Bayesian-powered probability insights to identify which SDRs were likely to succeed — and which might need additional support.


# Forecasting and Bayesian Model

This section focuses on the final data used in the Bayesian model and a preview of select reps' performance probabilities.

Displayed DataFrame: `rep_contribution_with_forecast`

Includes:
- `rep_expected_sales_q3`
- `rep_weighted_avg_sales`
- `std_weekly_sales`
- `sales_contribution_ratio`

In [107]:
# ---------------------------------------------
# Get Most Recent Quarter (Q2) Actuals at Territory Level
# ---------------------------------------------
recent_actuals = territory_performance_validation[[
    'territory_id', 'recent_quarter',
    'recent_deals', 'recent_wins', 'recent_win_rate',
    'recent_amount', 'recent_weekly_sales'
]].rename(columns={
    'recent_quarter': 'previous_quarter',
    'recent_deals': 'prev_deals',
    'recent_wins': 'prev_wins',
    'recent_win_rate': 'prev_win_rate',
    'recent_amount': 'prev_amount',
    'recent_weekly_sales': 'prev_weekly_sales'
})

# ---------------------------------------------
# Merge Territory Forecast with Actuals
# ---------------------------------------------
comparison_df = pd.merge(
    territory_forecast,
    recent_actuals,
    on='territory_id',
    how='left'
)

# ---------------------------------------------
# Calculate % Change from Q2 to Q3
# ---------------------------------------------
comparison_df['deals_pct_change'] = (comparison_df['forecast_deals'] - comparison_df['prev_deals']) / comparison_df['prev_deals']
comparison_df['wins_pct_change'] = (comparison_df['forecast_wins'] - comparison_df['prev_wins']) / comparison_df['prev_wins']
comparison_df['sales_pct_change'] = (comparison_df['forecast_sales'] - comparison_df['prev_weekly_sales']) / comparison_df['prev_weekly_sales']
comparison_df['amount_pct_change'] = (comparison_df['forecast_amount'] - comparison_df['prev_amount']) / comparison_df['prev_amount']

# ---------------------------------------------
# Get Rep-Level Sales Contribution (Q2)
# ---------------------------------------------
q2_rep_contributions = (
    rep_territory_quarter_summary[
        rep_territory_quarter_summary['year_quarter'] == '2015-Q2'
    ][['account_owned_by', 'territory_id', 'total_weekly_sales']]
    .groupby(['territory_id', 'account_owned_by'], as_index=False)
    .agg(rep_sales_q2=('total_weekly_sales', 'sum'))
)


# ---------------------------------------------
# Merge Rep Contributions with Territory Comparison
# ---------------------------------------------
rep_contribution_with_forecast = pd.merge(
    q2_rep_contributions,
    comparison_df[['territory_id', 'forecast_quarter', 'forecast_sales', 'prev_weekly_sales']],
    on='territory_id',
    how='left'
)

# ---------------------------------------------
# Calculate % Contribution + Rep-Level Q3 Forecast
# ---------------------------------------------
rep_contribution_with_forecast['rep_sales_contribution_ratio'] = (
    rep_contribution_with_forecast['rep_sales_q2'] / rep_contribution_with_forecast['prev_weekly_sales']
)

rep_contribution_with_forecast['rep_expected_sales_q3'] = (
    rep_contribution_with_forecast['forecast_sales'] * rep_contribution_with_forecast['rep_sales_contribution_ratio']
)

# ---------------------------------------------
# Merge Rep Weighted Avg into Contribution + Forecast
# ---------------------------------------------
rep_contribution_with_forecast = pd.merge(
    rep_contribution_with_forecast,
    rep_weighted_avg[['account_owned_by', 'territory_id', 'rep_weighted_avg_sales']],
    on=['account_owned_by', 'territory_id'],
    how='left'
)

# ---------------------------------------------
# Sort & Preview
# ---------------------------------------------
rep_contribution_with_forecast = rep_contribution_with_forecast.sort_values(by=['account_owned_by', 'territory_id'])

display(rep_contribution_with_forecast.head(10))




,territory_id,account_owned_by,rep_sales_q2,forecast_quarter,forecast_sales,prev_weekly_sales,rep_sales_contribution_ratio,rep_expected_sales_q3,rep_weighted_avg_sales
29,9,david ward,176982.0,2015-Q3,376826.22,420431.0,0.420954,158626.404970,85891.920000
0,1,deepak ramirez,71262.0,2015-Q3,154555.89,151366.2,0.470792,72763.680618,69810.390000
4,2,deepak ramirez,0.0,2015-Q3,119520.70,94876.0,0.000000,0.000000,34396.700000
8,3,deepak ramirez,91344.0,2015-Q3,1066682.14,1211646.4,0.075388,80415.386367,97419.885000
1,1,dennis watson,16020.0,2015-Q3,154555.89,151366.2,0.105836,16357.584175,36459.936000
5,2,dennis watson,24536.0,2015-Q3,119520.70,94876.0,0.258611,30909.396425,128961.385714
9,3,dennis watson,100898.4,2015-Q3,1066682.14,1211646.4,0.083274,88826.675204,151324.010000
39,13,elissa sanders,80502.0,2015-Q3,424252.47,487278.6,0.165207,70089.620886,72589.440000
43,14,elissa sanders,47740.0,2015-Q3,165971.45,133271.0,0.358217,59453.872358,89281.162500
47,15,elissa sanders,150316.8,2015-Q3,1064722.94,1166824.0,0.128826,137163.569851,176938.578182


In [105]:
# Fallback std deviation if none exists
fallback_std_ratio = 0.25

# Storage for results
bayes_results = []

# ---------------------------------------------
# Filter to single rep–territory row
# ---------------------------------------------
rep_row = rep_contribution_with_forecast[
    (rep_contribution_with_forecast['account_owned_by'] == 'elissa sanders') &
    (rep_contribution_with_forecast['territory_id'] == 14)
].copy()

if not rep_row.empty:
    row = rep_row.reset_index(drop=True).iloc[0]
    mu = row['rep_weighted_avg_sales']
    expected_sales_q3 = row['rep_expected_sales_q3']
    std = row.get('std_weekly_sales', np.nan)

    # Handle missing or zero values early
    if pd.isna(mu) or pd.isna(expected_sales_q3) or expected_sales_q3 == 0:
        result = {
            'posterior_mean_sales': np.nan,
            'prob_meet_or_exceed': np.nan,
            'credible_interval_lower': np.nan,
            'credible_interval_upper': np.nan,
            'sales_pressure_ratio': np.nan,
            'risk_flag': 'insufficient data'
        }
    else:
        if pd.isna(std) or std == 0:
            std = mu * fallback_std_ratio

        with pm.Model():
            sales = pm.Normal("sales", mu=mu, sigma=std)
            trace = pm.sample(2000, tune=1000, chains=2, progressbar=False)
            sales_samples = trace.posterior['sales'].stack(draws=("chain", "draw")).values

        posterior_mean = np.mean(sales_samples)
        prob_hit = np.mean(sales_samples >= expected_sales_q3)
        lower, upper = np.percentile(sales_samples, [3, 97])
        pressure_ratio = expected_sales_q3 / mu

        # Risk category
        if prob_hit < 0.33:
            risk_flag = 'high risk'
        elif prob_hit < 0.66:
            risk_flag = 'moderate risk'
        else:
            risk_flag = 'low risk'

        result = {
            'posterior_mean_sales': posterior_mean,
            'prob_meet_or_exceed': prob_hit,
            'credible_interval_lower': lower,
            'credible_interval_upper': upper,
            'sales_pressure_ratio': pressure_ratio,
            'risk_flag': risk_flag
        }

    # Append result
    bayes_results.append(result)

    # Combine with original row
    single_bayes_output = pd.concat([rep_row.reset_index(drop=True), pd.DataFrame(bayes_results)], axis=1)

    # Preview
    display(single_bayes_output)
else:
    print("No matching row found for that rep and territory.")

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sales]
Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 8 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


,territory_id,account_owned_by,rep_sales_q2,forecast_quarter,forecast_sales,prev_weekly_sales,rep_sales_contribution_ratio,rep_expected_sales_q3,rep_weighted_avg_sales,posterior_mean_sales,prob_meet_or_exceed,credible_interval_lower,credible_interval_upper,sales_pressure_ratio,risk_flag
0,14,elissa sanders,47740.0,2015-Q3,165971.45,133271.0,0.358217,59453.872358,89281.1625,89564.779304,0.91525,47977.323752,132676.613754,0.665917,low risk


# Appendix: Full Workflow

This section includes the full end-to-end Python pipeline I used in developing the forecast and model

Sections:
- Data Cleaning & Final Check
- Feature Engineering
- Performance Summaries
- Modeling
- Bayesian Model

All code has been executed and verified for:
- Consistency across logic
- Rolling temporal forecasting
- Realistic cap limits
- Scalable contribution logic per territory

In [1]:
# -----------------------------
# Import Libraries
# -----------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymc as pm
import pytensor
import pytensor.tensor as at
import arviz as az

# -----------------------------
# Load Datasets
# -----------------------------
deals_raw = pd.read_csv('salesforce data.csv')  

# -----------------------------
# Create Working Copies
# -----------------------------
deals_df = deals_raw.copy()

# -----------------------------
# Normalize Column Names
# -----------------------------
deals_df.columns = deals_df.columns.str.lower().str.strip().str.replace(' ', '_', regex=False)

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install gxx`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.


---

## Clean + Prep Salesforce Data

In [4]:
# -----------------------------
# Unique Value Preview (Max 5 per column)
# -----------------------------
print("Previewing Unique Values Per Column (Max 5 each):\n")

for col in deals_df.columns:
    unique_vals = deals_df[col].dropna().unique()
    sample_vals = unique_vals[:5] if len(unique_vals) > 5 else unique_vals
    print(f"{col} → {len(unique_vals)} unique values")
    print(f"Sample: {sample_vals}\n")


Previewing Unique Values Per Column (Max 5 each):

account_id → 709 unique values
Sample: ['001i0000008rFxEAAU' '001i0000008rFoZAAU' '001i0000008rG6VAAU'
 '001i0000008rG2oAAE' '001i0000008rCFaAAM']

account_name → 696 unique values
Sample: ['123 Warehousing' 'Mollis LLC' 'Global Phone Networks(GPN)'
 'Nec Consulting' 'Arcu PC']

account_owned_by → 19 unique values
Sample: ['Tara Wang' 'Fred Jenkins' 'Elissa Sanders' 'Greg Long'
 'Jennifer Gonzales']

account_type → 4 unique values
Sample: ['Gold' 'Other' 'Cloud' 'Platinum']

address → 709 unique values
Sample: ['3270 N Hwy 87, Pine, AZ 85544' '4746 E. Grant Rd., Tucson, AZ 85712'
 '110 N Magnolia Ave, El Cajon, CA 92020'
 '4101 30th St, San Diego, CA 92104'
 '40827 Stone Rd, Big Bear Lake, CA 92315']

billing_city → 494 unique values
Sample: ['Pine' 'Tucson' 'El Cajon' 'San Diego' 'Big Bear Lake']

billing_region → 5 unique values
Sample: ['Southwest' 'West' 'Southeast' 'Midwest' 'Northeast']

billing_state → 48 unique values
Sample: [

In [37]:
# -----------------------------
# Filter Relevant Columns
# -----------------------------

selected_cols = [
    'account_owned_by',
    'billing_region',
    'close_date',
    'closed',
    'opportunity_id',
    'opportunity_type',
    'product_name',
    'stage',
    'won',
    'amount',
    'weekly_sales'
]

deals_clean = deals_df[selected_cols].copy() 
print("New working DataFrame created with relevant columns only.")
display(deals_clean.head()) 

# -----------------------------
# Check current types
# -----------------------------
print("Data Types BEFORE Fixes:")
print(deals_clean.dtypes)

New working DataFrame created with relevant columns only.


,account_owned_by,billing_region,close_date,closed,opportunity_id,opportunity_type,product_name,stage,won,amount,weekly_sales
0,Tara Wang,Southwest,1/8/2011,True,006i0000001vt8dAAA,Maintenance,MOL Standard,Closed Lost,False,196.8,$0.0
1,Tara Wang,Southwest,1/10/2011,True,006i0000001vt8VAAQ,Software,MOL Standard,Closed Lost,False,21312.0,$0.0
2,Fred Jenkins,West,1/16/2011,True,006i0000001vt8pAAA,Maintenance,MOL Standard,Closed Won,True,1932.0,"$1,932.0"
3,Fred Jenkins,West,1/19/2011,True,006i0000001vt8yAAA,Software,MOL Standard,Closed Won,True,32536.0,"$32,536.0"
4,Fred Jenkins,West,1/20/2011,True,006i0000001vt8zAAA,Software,MOL Standard,Closed Won,True,2184.0,"$2,184.0"


Data Types BEFORE Fixes:
account_owned_by     object
billing_region       object
close_date           object
closed                 bool
opportunity_id       object
opportunity_type     object
product_name         object
stage                object
won                    bool
amount              float64
weekly_sales         object
dtype: object


In [39]:
# --------------------------------------
# Data Type Cleanup & Conversion
# --------------------------------------

# Convert close_date to datetime
deals_clean['close_date'] = pd.to_datetime(deals_clean['close_date'], errors='coerce')

# Parse monetary values from weekly_sales
def parse_money(x):
    if isinstance(x, str):
        x = x.replace('$', '').replace(',', '').strip()
        try:
            return float(x)
        except:
            return np.nan
    return x  # already numeric

deals_clean['weekly_sales'] = deals_clean['weekly_sales'].apply(parse_money)

# Final data type check
print("\n Data Types AFTER Conversion:")
print(deals_clean.dtypes)

# Optional: quick descriptive check on monetary field
print("\n Weekly Sales Distribution:")
print(deals_clean['weekly_sales'].describe())



 Data Types AFTER Conversion:
account_owned_by            object
billing_region              object
close_date          datetime64[ns]
closed                        bool
opportunity_id              object
opportunity_type            object
product_name                object
stage                       object
won                           bool
amount                     float64
weekly_sales               float64
dtype: object

 Weekly Sales Distribution:
count      3215.000000
mean      13881.198942
std       41310.315065
min           0.000000
25%           0.000000
50%        2215.200000
75%       10520.000000
max      663264.000000
Name: weekly_sales, dtype: float64


In [41]:
# -----------------------------
# Normalize String Column Values
# -----------------------------
string_cols = deals_clean.select_dtypes(include='object').columns

for col in string_cols:
    deals_clean[col] = deals_clean[col].str.lower().str.strip()

print("All string columns normalized to lowercase + trimmed.")
display(deals_clean[string_cols].sample(5))  # Optional preview


All string columns normalized to lowercase + trimmed.


,account_owned_by,billing_region,opportunity_id,opportunity_type,product_name,stage
1867,fred jenkins,west,006i0000001vwk2aai,maintenance,mol mobile,closed won
2718,jesse alexander,midwest,006i0000001vx5paaa,software,mol standard,closed won
1442,jason washington,southeast,006i0000001vwuxaai,services,mol standard,closed lost
2172,jamie data,northeast,006i0000001vwyyaai,software,mol premium,closed won
641,kevin gibson,northeast,006i0000001vuaqaai,software,mol standard,closed won


In [43]:
# -----------------------------
# Total Rows
# -----------------------------
print(f"Total Rows in deals_clean: {len(deals_clean)}")

# -----------------------------
# Unique Values (Capped at 7) — All Columns
# -----------------------------
print("Unique Value Samples:")
for col in deals_clean.columns:
    unique_vals = deals_clean[col].dropna().unique()
    print(f"\n{col} → {len(unique_vals)} unique values")
    print("Sample:", unique_vals[:7])

# -----------------------------
# Null Count Per Column
# -----------------------------
print("\n Null Count:")
print(deals_clean.isnull().sum().sort_values(ascending=False))

# -----------------------------
# Duplicate Checks
# -----------------------------
print("\n Full Row Duplicates:", deals_clean.duplicated().sum())
print(" Duplicate Opportunity IDs:", deals_clean['opportunity_id'].duplicated().sum())

# -----------------------------
# Data Type Overview
# -----------------------------
print("\n Data Type Counts:")
print(deals_clean.dtypes.value_counts())

# -----------------------------
# Summary Stats for Numeric Fields
# -----------------------------
print("\n Numeric Summary:")
display(deals_clean.describe(include=[np.number]).T)


Total Rows in deals_clean: 3215
Unique Value Samples:

account_owned_by → 19 unique values
Sample: ['tara wang' 'fred jenkins' 'elissa sanders' 'greg long'
 'jennifer gonzales' 'jesse alexander' 'deepak ramirez']

billing_region → 5 unique values
Sample: ['southwest' 'west' 'southeast' 'midwest' 'northeast']

close_date → 1186 unique values
Sample: <DatetimeArray>
['2011-01-08 00:00:00', '2011-01-10 00:00:00', '2011-01-16 00:00:00',
 '2011-01-19 00:00:00', '2011-01-20 00:00:00', '2011-01-22 00:00:00',
 '2011-01-30 00:00:00']
Length: 7, dtype: datetime64[ns]

closed → 2 unique values
Sample: [ True False]

opportunity_id → 3215 unique values
Sample: ['006i0000001vt8daaa' '006i0000001vt8vaaq' '006i0000001vt8paaa'
 '006i0000001vt8yaaa' '006i0000001vt8zaaa' '006i0000001vt8uaaa'
 '006i0000001vt9jaaq']

opportunity_type → 3 unique values
Sample: ['maintenance' 'software' 'services']

product_name → 3 unique values
Sample: ['mol standard' 'mol mobile' 'mol premium']

stage → 7 unique values
S

,count,mean,std,min,25%,50%,75%,max
amount,3215.0,24303.621182,61200.848186,85.2,1995.0,6776.0,17100.0,1181565.0
weekly_sales,3215.0,13881.198942,41310.315065,0.0,0.0,2215.2,10520.0,663264.0


In [14]:
# -----------------------------
# Logical Validations Across Status Columns
# -----------------------------

# Closed == True should only have stage of Closed Won or Closed Lost
valid_closed_stages = ['closed won', 'closed lost']
closed_mismatch = deals_clean[
    (deals_clean['closed'] == True) &
    (~deals_clean['stage'].isin(valid_closed_stages))
]
print(f" Closed == True but invalid stage: {len(closed_mismatch)} rows")

# Closed == False should only have stage of in-progress values
in_progress_stages = ['solution', 'commit', 'decision', 'discover', 'qualify']
open_mismatch = deals_clean[
    (deals_clean['closed'] == False) &
    (~deals_clean['stage'].isin(in_progress_stages))
]
print(f" Closed == False but stage is not an open stage: {len(open_mismatch)} rows")

# won == True should match stage == Closed Won
won_flag_mismatch = deals_clean[
    (deals_clean['won'] == True) & (deals_clean['stage'] != 'closed won')
]
print(f" Won == True but stage != 'closed won': {len(won_flag_mismatch)} rows")

# stage == Closed Won should match won == True
stage_flag_mismatch = deals_clean[
    (deals_clean['stage'] == 'closed won') & (deals_clean['won'] != True)
]
print(f" Stage == 'closed won' but won flag != True: {len(stage_flag_mismatch)} rows")

# Summary Check
closed_true_count = deals_clean['closed'].sum()
closed_stage_count = deals_clean['stage'].isin(['closed won', 'closed lost']).sum()
print(f"\n Total closed == True: {closed_true_count}")
print(f" Total deals with stage Closed Won/Lost: {closed_stage_count}")
print(f" Do they match? → {closed_true_count == closed_stage_count}")

# Check for near-duplicates (same rep, region, date, product)
potential_duplicates = deals_clean[
    deals_clean.duplicated(subset=['account_owned_by', 'billing_region', 'close_date', 'product_name'], keep=False)
]
print(f"\n Potential duplicates by rep + region + date + product: {len(potential_duplicates)}")
if len(potential_duplicates):
    display(potential_duplicates.sort_values(by=['account_owned_by', 'close_date']).head(5))

# Confirm unique opportunity_id
dupe_ids = deals_clean['opportunity_id'].duplicated().sum()
print(f"\n Duplicate opportunity_id count: {dupe_ids} (should be 0)")

 Closed == True but invalid stage: 6 rows
 Closed == False but stage is not an open stage: 0 rows
 Won == True but stage != 'closed won': 0 rows
 Stage == 'closed won' but won flag != True: 0 rows

 Total closed == True: 3055
 Total deals with stage Closed Won/Lost: 3049
 Do they match? → False

 Potential duplicates by rep + region + date + product: 705


,account_owned_by,billing_region,close_date,closed,opportunity_id,opportunity_type,product_name,stage,won,amount,weekly_sales
484,david ward,southeast,2012-10-27,True,006i0000001vtxuaaq,maintenance,mol standard,closed won,True,2678.4,2678.4
485,david ward,southeast,2012-10-27,True,006i0000001vtxtaaq,software,mol standard,closed won,True,55760.0,55760.0
1267,david ward,southeast,2013-12-22,True,006i0000001vv9caaq,software,mol premium,closed lost,False,3528.0,0.0
2183,david ward,southeast,2013-12-22,True,006i0000001vv7caaa,software,mol premium,closed won,True,1896.0,1896.0
1195,david ward,southeast,2013-12-26,True,006i0000001vvalaay,software,mol mobile,closed lost,False,864.0,0.0



 Duplicate opportunity_id count: 0 (should be 0)


In [45]:
# -----------------------------
# Visual Check: Closed True but Stage Not Closed Won/Lost
# -----------------------------
invalid_stage_rows = deals_clean[
    (deals_clean['closed'] == True) &
    (~deals_clean['stage'].isin(['closed won', 'closed lost']))
][[
    'account_owned_by', 'billing_region', 'close_date',
    'opportunity_type', 'product_name', 'stage', 'closed', 'won', 
    'amount', 'weekly_sales', 'opportunity_id'
]]

print(f" Closed == True but invalid stage: {len(invalid_stage_rows)} rows")
display(invalid_stage_rows)

 Closed == True but invalid stage: 6 rows


,account_owned_by,billing_region,close_date,opportunity_type,product_name,stage,closed,won,amount,weekly_sales,opportunity_id
1194,kevin gibson,northeast,2015-02-23,software,mol mobile,discover,True,False,13536.0,0.0,006i0000001vxb0aai
1291,elissa sanders,west,2015-06-20,software,mol premium,solution,True,False,7488.0,0.0,006i0000001vxx8aaa
1329,keyur cruz,midwest,2015-05-03,maintenance,mol standard,decision,True,False,1029.6,0.0,006i0000001vxfaaaa
1411,deepak ramirez,midwest,2013-09-20,services,mol standard,discover,True,False,497.0,0.0,006i0000001vunwaaq
1412,jesse alexander,midwest,2015-04-13,services,mol standard,solution,True,False,10507.0,0.0,006i0000001vxwbaai
1453,jason washington,southeast,2014-09-20,services,mol standard,solution,True,False,504.0,0.0,006i0000001vwkcaai


In [47]:
# -----------------------------
# Closed == True, Won == False, Weekly Sales == 0 → Stage = 'closed lost'
# -----------------------------
fix_condition = (
    (deals_clean['closed'] == True) &
    (deals_clean['won'] == False) &
    (deals_clean['weekly_sales'] == 0)
)

# Apply fix
deals_clean.loc[fix_condition, 'stage'] = 'closed lost'

# Re-validate
# Closed == True should only have stage of Closed Won or Closed Lost
valid_closed_stages = ['closed won', 'closed lost']
closed_mismatch = deals_clean[
    (deals_clean['closed'] == True) &
    (~deals_clean['stage'].isin(valid_closed_stages))
]
print(f" Closed == True but invalid stage: {len(closed_mismatch)} rows")

# Closed == False should only have stage of in-progress values
in_progress_stages = ['solution', 'commit', 'decision', 'discover', 'qualify']
open_mismatch = deals_clean[
    (deals_clean['closed'] == False) &
    (~deals_clean['stage'].isin(in_progress_stages))
]
print(f" Closed == False but stage is not an open stage: {len(open_mismatch)} rows")

# won == True should match stage == Closed Won
won_flag_mismatch = deals_clean[
    (deals_clean['won'] == True) & (deals_clean['stage'] != 'closed won')
]
print(f" Won == True but stage != 'closed won': {len(won_flag_mismatch)} rows")

# stage == Closed Won should match won == True
stage_flag_mismatch = deals_clean[
    (deals_clean['stage'] == 'closed won') & (deals_clean['won'] != True)
]
print(f" Stage == 'closed won' but won flag != True: {len(stage_flag_mismatch)} rows")

 Closed == True but invalid stage: 0 rows
 Closed == False but stage is not an open stage: 0 rows
 Won == True but stage != 'closed won': 0 rows
 Stage == 'closed won' but won flag != True: 0 rows


In [49]:
# -----------------------------
# Win/Loss Integrity Checks
# -----------------------------

print("Behavioral Validation Summary:\n")

# Won deals should NOT have amount == 0
bad_amount_won = deals_clean[(deals_clean['won']) & (deals_clean['amount'] <= 0)]
print(f" Won deals with zero or negative amount: {len(bad_amount_won)}")

# Won deals should NOT have weekly_sales == 0
bad_sales_won = deals_clean[(deals_clean['won']) & (deals_clean['weekly_sales'] <= 0)]
print(f" Won deals with zero or negative weekly_sales: {len(bad_sales_won)}")

# Lost deals should NOT have weekly_sales > 0
bad_sales_lost = deals_clean[
    (deals_clean['stage'] == 'closed lost') & (deals_clean['weekly_sales'] > 0)
]
print(f" Lost deals with positive weekly_sales: {len(bad_sales_lost)}")

# Lost deals should NOT have amount > 0
bad_amount_lost = deals_clean[
    (deals_clean['stage'] == 'closed lost') & (deals_clean['amount'] > 0)
]
print(f" Lost deals with positive amount: {len(bad_amount_lost)}")

# Weekly sales should not exceed amount
exceeds_revenue = deals_clean[deals_clean['weekly_sales'] > deals_clean['amount']]
print(f" Weekly sales greater than total amount: {len(exceeds_revenue)}")

# Won deals with stage != 'closed won'
won_bad_stage = deals_clean[(deals_clean['won']) & (deals_clean['stage'] != 'closed won')]
print(f" Won flag true but stage != 'closed won': {len(won_bad_stage)}")

# Stage 'closed won' but won flag is not True
stage_won_bad_flag = deals_clean[(deals_clean['stage'] == 'closed won') & (~deals_clean['won'])]
print(f" Stage == 'closed won' but won != True: {len(stage_won_bad_flag)}")

# Closed == True but weekly_sales still 0 — suspicious if not lost
closed_zero_sales = deals_clean[
    (deals_clean['closed']) & 
    (deals_clean['weekly_sales'] == 0) & 
    (deals_clean['stage'] != 'closed lost')
]
print(f" Closed deals with zero weekly_sales but not marked as 'closed lost': {len(closed_zero_sales)}")

# Any deals where amount is null or missing
null_amounts = deals_clean['amount'].isnull().sum()
print(f" Null or missing amount values: {null_amounts}")

# Any deals where weekly_sales is null or missing
null_sales = deals_clean['weekly_sales'].isnull().sum()
print(f" Null or missing weekly_sales values: {null_sales}")

Behavioral Validation Summary:

 Won deals with zero or negative amount: 0
 Won deals with zero or negative weekly_sales: 0
 Lost deals with positive weekly_sales: 0
 Lost deals with positive amount: 935
 Weekly sales greater than total amount: 0
 Won flag true but stage != 'closed won': 0
 Stage == 'closed won' but won != True: 0
 Closed deals with zero weekly_sales but not marked as 'closed lost': 0
 Null or missing amount values: 0
 Null or missing weekly_sales values: 0


---

## Final Check 

In [52]:
# -----------------------------
# Final Visual Sanity Check: Columns, Casing, Unique Values, and Ranges
# -----------------------------

print(" Column Names (Final Check for Casing + Underscore Format):")
print(deals_clean.columns.tolist())

print("\n Unique Values per Column (Capped at 7):")
for col in deals_clean.columns:
    print(f"\n--- {col} ({deals_clean[col].dtype}) ---")
    uniques = deals_clean[col].dropna().unique()
    sample = uniques[:7]
    print(f"Unique count: {len(uniques)}")
    print("Sample:", sample)

# -----------------------------
# Ranges for Numeric & Date Fields
# -----------------------------
print("\n Min/Max Ranges for Numeric & Date Columns:")
for col in deals_clean.select_dtypes(include=['float64', 'int64', 'datetime64[ns]']):
    col_min = deals_clean[col].min()
    col_max = deals_clean[col].max()
    print(f"{col}: Min = {col_min}, Max = {col_max}")

 Column Names (Final Check for Casing + Underscore Format):
['account_owned_by', 'billing_region', 'close_date', 'closed', 'opportunity_id', 'opportunity_type', 'product_name', 'stage', 'won', 'amount', 'weekly_sales']

 Unique Values per Column (Capped at 7):

--- account_owned_by (object) ---
Unique count: 19
Sample: ['tara wang' 'fred jenkins' 'elissa sanders' 'greg long'
 'jennifer gonzales' 'jesse alexander' 'deepak ramirez']

--- billing_region (object) ---
Unique count: 5
Sample: ['southwest' 'west' 'southeast' 'midwest' 'northeast']

--- close_date (datetime64[ns]) ---
Unique count: 1186
Sample: <DatetimeArray>
['2011-01-08 00:00:00', '2011-01-10 00:00:00', '2011-01-16 00:00:00',
 '2011-01-19 00:00:00', '2011-01-20 00:00:00', '2011-01-22 00:00:00',
 '2011-01-30 00:00:00']
Length: 7, dtype: datetime64[ns]

--- closed (bool) ---
Unique count: 2
Sample: [ True False]

--- opportunity_id (object) ---
Unique count: 3215
Sample: ['006i0000001vt8daaa' '006i0000001vt8vaaq' '006i0000001

---

## Feature Engineering 

In [55]:
# ------------------------------------------------
# Temporal Fields & Territory ID
# ------------------------------------------------

# Temporal Features
deals_clean['year'] = deals_clean['close_date'].dt.year
deals_clean['quarter'] = deals_clean['close_date'].dt.quarter
deals_clean['year_quarter'] = deals_clean['year'].astype(str) + '-Q' + deals_clean['quarter'].astype(str)
deals_clean['year_month'] = deals_clean['close_date'].dt.to_period('M').astype(str)

# lean group fields just in case
deals_clean['billing_region'] = deals_clean['billing_region'].str.strip().str.lower()
deals_clean['product_name'] = deals_clean['product_name'].str.strip().str.lower()

# Define Territory ID based on billing_region + product_name
group_fields = ['billing_region', 'product_name']
territory_lookup = (
    deals_clean
    .drop_duplicates(subset=group_fields)
    .sort_values(by=group_fields)
    .reset_index(drop=True)
    .loc[:, group_fields]
    .copy()
)
territory_lookup['territory_id'] = range(1, len(territory_lookup) + 1)

# Merge back into main DataFrame
deals_clean = deals_clean.merge(territory_lookup, on=group_fields, how='left')

# Final QA
print("Unique Temporary Territory IDs:", deals_clean['territory_id'].nunique())

display(deals_clean['territory_id'].value_counts().sort_index())

print("Merge successful. Null territory_id count:", deals_clean['territory_id'].isnull().sum())
display(deals_clean[['billing_region', 'product_name', 'territory_id']].drop_duplicates().sort_values(by='territory_id').head(10))

Unique Temporary Territory IDs: 15


territory_id
1      85
2      90
3     643
4      81
5      82
6     698
7      35
8      34
9     337
10     28
11     33
12    199
13    130
14    102
15    638
Name: count, dtype: int64

Merge successful. Null territory_id count: 0


,billing_region,product_name,territory_id
20,midwest,mol mobile,1
532,midwest,mol premium,2
18,midwest,mol standard,3
67,northeast,mol mobile,4
113,northeast,mol premium,5
30,northeast,mol standard,6
352,southeast,mol mobile,7
420,southeast,mol premium,8
11,southeast,mol standard,9
114,southwest,mol mobile,10


---

## Performance Summaries

In [58]:
# ------------------------------------------------
# Territory-Level Quarterly Summary
# ------------------------------------------------

# Only use closed deals (won or lost)
valid_deals = deals_clean[deals_clean['stage'].isin(['closed won', 'closed lost'])].copy()
valid_deals['is_won'] = valid_deals['stage'] == 'closed won'

# Aggregate performance by territory and year_quarter
territory_quarter_summary = (
    valid_deals
    .groupby(['territory_id', 'year_quarter'])
    .agg(
        total_deals=('opportunity_id', 'count'),
        total_wins=('is_won', 'sum'),
        win_rate=('is_won', 'mean'),
        total_amount=('amount', 'sum'),
        total_weekly_sales=('weekly_sales', 'sum')
    )
    .reset_index()
    .sort_values(by=['territory_id', 'year_quarter'])
)

# Preview the first few rows
print("Territory-Level Performance by Quarter:")
display(territory_quarter_summary.head(10))

Territory-Level Performance by Quarter:


,territory_id,year_quarter,total_deals,total_wins,win_rate,total_amount,total_weekly_sales
0,1,2011-Q1,1,1,1.000000,961.2,961.2
1,1,2012-Q4,2,2,1.000000,2280.0,2280.0
2,1,2013-Q1,4,3,0.750000,10772.4,10250.4
3,1,2013-Q2,3,2,0.666667,68959.8,68203.8
4,1,2013-Q3,8,5,0.625000,511056.0,33346.8
5,1,2013-Q4,6,5,0.833333,36447.6,34971.6
6,1,2014-Q1,5,3,0.600000,71253.0,48960.0
7,1,2014-Q2,11,10,0.909091,445549.2,442921.2
8,1,2014-Q3,7,3,0.428571,55668.0,49662.0
9,1,2014-Q4,16,16,1.000000,421008.0,421008.0


In [60]:
# ------------------------------------------------
# Group & Summarize Last 8 + Most Recent Quarter (No Deprecation Warning)
# ------------------------------------------------

# Remove grouping column before apply
grouped = territory_quarter_summary.drop(columns='territory_id').copy()
territory_ids = territory_quarter_summary['territory_id'].unique()

# Apply groupby safely without warning
summary_list = []

for tid in territory_ids:
    sub = territory_quarter_summary[territory_quarter_summary['territory_id'] == tid]
    sub = sub.sort_values(by='year_quarter')

    last_8 = sub.tail(8)
    recent = sub.tail(1)

    summary_list.append({
        'territory_id': tid,
        'avg_deals': last_8['total_deals'].mean(),
        'avg_wins': last_8['total_wins'].mean(),
        'avg_win_rate': last_8['win_rate'].mean(),
        'avg_amount': last_8['total_amount'].mean(),
        'avg_weekly_sales': last_8['total_weekly_sales'].mean(),

        'recent_quarter': recent['year_quarter'].values[0],
        'recent_deals': recent['total_deals'].values[0],
        'recent_wins': recent['total_wins'].values[0],
        'recent_win_rate': recent['win_rate'].values[0],
        'recent_amount': recent['total_amount'].values[0],
        'recent_weekly_sales': recent['total_weekly_sales'].values[0]
    })

# Convert to DataFrame
territory_performance_validation = pd.DataFrame(summary_list)

# Preview
display(territory_performance_validation)

,territory_id,avg_deals,avg_wins,avg_win_rate,avg_amount,avg_weekly_sales,recent_quarter,recent_deals,recent_wins,recent_win_rate,recent_amount,recent_weekly_sales
0,1,8.750,7.125,0.769142,2.247098e+05,159340.4250,2015-Q2,7,6,0.857143,152113.2,151366.2
1,2,9.750,7.375,0.728943,1.921621e+05,156487.7500,2015-Q2,4,4,1.000000,94876.0,94876.0
2,3,59.375,37.750,0.642293,1.369294e+06,849235.7500,2015-Q2,46,34,0.739130,1345072.6,1211646.4
3,4,6.125,4.500,0.691667,1.281234e+05,103166.4000,2015-Q2,4,2,0.500000,73008.0,9540.0
4,5,7.375,5.750,0.649053,1.652318e+05,140793.4375,2015-Q2,11,10,0.909091,260488.0,218440.0
5,6,42.875,26.125,0.614810,1.095147e+06,656807.6000,2015-Q2,25,14,0.560000,656501.6,410674.6
6,7,3.500,2.250,0.643750,1.026478e+05,38059.2750,2015-Q2,4,4,1.000000,90392.4,90392.4
7,8,4.000,2.625,0.755208,2.238932e+05,140811.0625,2015-Q2,4,2,0.500000,189408.0,182676.0
8,9,24.500,15.875,0.654378,6.371691e+05,311419.0500,2015-Q2,20,12,0.600000,478244.0,420431.0
9,10,3.250,2.250,0.708333,5.154818e+04,24568.9500,2015-Q2,4,1,0.250000,99180.0,697.2


In [62]:
# -----------------------------
# Check for 2015 Q3 Deals
# -----------------------------

# Ensure year_quarter column is accurate
deals_clean['year'] = deals_clean['close_date'].dt.year
deals_clean['quarter'] = deals_clean['close_date'].dt.quarter
deals_clean['year_quarter'] = deals_clean['year'].astype(str) + '-Q' + deals_clean['quarter'].astype(str)

# Filter for 2015-Q3
q3_2015_deals = deals_clean[deals_clean['year_quarter'] == '2015-Q3']
print(f" Total Deals in 2015-Q3: {len(q3_2015_deals)}")

# Stage distribution
print("\n Stage Distribution in 2015-Q3:")
print(q3_2015_deals['stage'].value_counts())

# Show won vs lost count
print("\n Won/Lost in 2015-Q3:")
print(q3_2015_deals['won'].value_counts())

# Show by territory if assigned
if 'territory_id' in deals_clean.columns:
    print("\n 2015-Q3 Deals by Territory:")
    print(q3_2015_deals['territory_id'].value_counts())

# Preview sample
display(q3_2015_deals[['account_owned_by', 'close_date', 'stage', 'won', 'amount', 'weekly_sales']].head(10))

 Total Deals in 2015-Q3: 24

 Stage Distribution in 2015-Q3:
stage
discover    11
decision     4
solution     3
commit       3
qualify      3
Name: count, dtype: int64

 Won/Lost in 2015-Q3:
won
False    24
Name: count, dtype: int64

 2015-Q3 Deals by Territory:
territory_id
15    6
3     5
14    3
13    2
4     2
6     2
1     1
8     1
2     1
12    1
Name: count, dtype: int64


,account_owned_by,close_date,stage,won,amount,weekly_sales
1000,deepak ramirez,2015-07-19,solution,False,21111.6,0.0
1007,eric barnes,2015-08-29,decision,False,79464.0,0.0
1017,eric barnes,2015-08-15,decision,False,68607.0,0.0
1021,greg powell,2015-08-23,commit,False,35532.0,0.0
1022,greg powell,2015-08-10,discover,False,810.0,0.0
1025,fred jenkins,2015-08-19,discover,False,1995.0,0.0
1027,joe ford,2015-08-16,decision,False,4272.0,0.0
1036,dennis watson,2015-07-23,qualify,False,189600.0,0.0
1038,eric barnes,2015-07-01,commit,False,491832.0,0.0
1042,eric barnes,2015-07-23,discover,False,107352.0,0.0


In [64]:
# ---------------------------------------------
# Forecast Next Quarter Quotas per Territory
# ---------------------------------------------

# Weights
w_recent = 0.6
w_avg = 0.4
cap_limit = 0.35  # Cap forecasts at ±35% of recent actuals

# Get most recent quarter
latest_q = territory_performance_validation['recent_quarter'].max()
latest_year, latest_q_num = latest_q.split('-Q')
latest_year = int(latest_year)
latest_q_num = int(latest_q_num)

# Advance to next quarter
if latest_q_num == 4:
    next_year = latest_year + 1
    next_q = 1
else:
    next_year = latest_year
    next_q = latest_q_num + 1

next_quarter_str = f"{next_year}-Q{next_q}"

# Copy and compute weighted forecasts
forecast_df = territory_performance_validation.copy()
forecast_df['forecast_quarter'] = next_quarter_str

forecast_df['forecast_deals'] = (
    w_recent * forecast_df['recent_deals'] + 
    w_avg * forecast_df['avg_deals']
)

forecast_df['forecast_wins'] = (
    w_recent * forecast_df['recent_wins'] + 
    w_avg * forecast_df['avg_wins']
)

forecast_df['forecast_win_rate'] = (
    w_recent * forecast_df['recent_win_rate'] + 
    w_avg * forecast_df['avg_win_rate']
)

forecast_df['forecast_amount'] = (
    w_recent * forecast_df['recent_amount'] + 
    w_avg * forecast_df['avg_amount']
)

forecast_df['forecast_sales'] = (
    w_recent * forecast_df['recent_weekly_sales'] + 
    w_avg * forecast_df['avg_weekly_sales']
)

# ---------------------------------------------
# Cap Forecasts to ±35% of Recent Actuals
# ---------------------------------------------
column_map = {
    'forecast_deals': 'recent_deals',
    'forecast_wins': 'recent_wins',
    'forecast_amount': 'recent_amount',
    'forecast_sales': 'recent_weekly_sales'
}

for forecast_col, recent_col in column_map.items():
    upper_cap = forecast_df[recent_col] * (1 + cap_limit)
    lower_cap = forecast_df[recent_col] * (1 - cap_limit)
    forecast_df[forecast_col] = forecast_df[forecast_col].clip(lower=lower_cap, upper=upper_cap)

# Final Output
territory_forecast = forecast_df[[
    'territory_id', 'forecast_quarter',
    'forecast_deals', 'forecast_wins', 'forecast_win_rate',
    'forecast_amount', 'forecast_sales'
]]

print(f" Forecasting for Quarter: {next_quarter_str}")
display(territory_forecast)

 Forecasting for Quarter: 2015-Q3


,territory_id,forecast_quarter,forecast_deals,forecast_wins,forecast_win_rate,forecast_amount,forecast_sales
0,1,2015-Q3,7.70,6.45,0.821943,181151.850,154555.890
1,2,2015-Q3,5.40,5.35,0.891577,128082.600,119520.700
2,3,2015-Q3,51.35,35.50,0.700396,1354761.300,1066682.140
3,4,2015-Q3,4.85,2.70,0.576667,95054.160,12879.000
4,5,2015-Q3,9.55,8.30,0.805076,222385.525,187381.375
5,6,2015-Q3,32.15,18.85,0.581924,831959.940,509127.800
6,7,2015-Q3,3.80,3.30,0.857500,95294.580,69459.150
7,8,2015-Q3,4.00,2.25,0.602083,203202.075,165930.025
8,9,2015-Q3,21.80,13.55,0.621751,541814.050,376826.220
9,10,2015-Q3,3.70,1.35,0.433333,80127.270,941.220


In [66]:
# Get most recent quarter actuals (Q2)
recent_actuals = territory_performance_validation[[
    'territory_id', 'recent_quarter',
    'recent_deals', 'recent_wins', 'recent_win_rate',
    'recent_amount', 'recent_weekly_sales'
]].rename(columns={
    'recent_quarter': 'previous_quarter',
    'recent_deals': 'prev_deals',
    'recent_wins': 'prev_wins',
    'recent_win_rate': 'prev_win_rate',
    'recent_amount': 'prev_amount',
    'recent_weekly_sales': 'prev_weekly_sales'
})

# Merge with forecast
comparison_df = pd.merge(
    territory_forecast,
    recent_actuals,
    on='territory_id',
    how='left'
)

# Calculate % changes
comparison_df['deals_pct_change'] = (comparison_df['forecast_deals'] - comparison_df['prev_deals']) / comparison_df['prev_deals']
comparison_df['wins_pct_change'] = (comparison_df['forecast_wins'] - comparison_df['prev_wins']) / comparison_df['prev_wins']
comparison_df['sales_pct_change'] = (comparison_df['forecast_sales'] - comparison_df['prev_weekly_sales']) / comparison_df['prev_weekly_sales']
comparison_df['amount_pct_change'] = (comparison_df['forecast_amount'] - comparison_df['prev_amount']) / comparison_df['prev_amount']

# Preview with formatting
cols_to_show = [
    'territory_id', 'forecast_quarter', 'previous_quarter',
    'forecast_deals', 'prev_deals', 'deals_pct_change',
    'forecast_wins', 'prev_wins', 'wins_pct_change',
    'forecast_sales', 'prev_weekly_sales', 'sales_pct_change',
    'forecast_amount', 'prev_amount', 'amount_pct_change'
]

# Show rounded for clarity
comparison_df[cols_to_show]

,territory_id,forecast_quarter,previous_quarter,forecast_deals,prev_deals,deals_pct_change,forecast_wins,prev_wins,wins_pct_change,forecast_sales,prev_weekly_sales,sales_pct_change,forecast_amount,prev_amount,amount_pct_change
0,1,2015-Q3,2015-Q2,7.70,7,0.100000,6.45,6,0.075000,154555.890,151366.2,0.021073,181151.850,152113.2,0.190902
1,2,2015-Q3,2015-Q2,5.40,4,0.350000,5.35,4,0.337500,119520.700,94876.0,0.259757,128082.600,94876.0,0.350000
2,3,2015-Q3,2015-Q2,51.35,46,0.116304,35.50,34,0.044118,1066682.140,1211646.4,-0.119642,1354761.300,1345072.6,0.007203
3,4,2015-Q3,2015-Q2,4.85,4,0.212500,2.70,2,0.350000,12879.000,9540.0,0.350000,95054.160,73008.0,0.301969
4,5,2015-Q3,2015-Q2,9.55,11,-0.131818,8.30,10,-0.170000,187381.375,218440.0,-0.142184,222385.525,260488.0,-0.146273
5,6,2015-Q3,2015-Q2,32.15,25,0.286000,18.85,14,0.346429,509127.800,410674.6,0.239735,831959.940,656501.6,0.267263
6,7,2015-Q3,2015-Q2,3.80,4,-0.050000,3.30,4,-0.175000,69459.150,90392.4,-0.231582,95294.580,90392.4,0.054232
7,8,2015-Q3,2015-Q2,4.00,4,0.000000,2.25,2,0.125000,165930.025,182676.0,-0.091670,203202.075,189408.0,0.072827
8,9,2015-Q3,2015-Q2,21.80,20,0.090000,13.55,12,0.129167,376826.220,420431.0,-0.103714,541814.050,478244.0,0.132924
9,10,2015-Q3,2015-Q2,3.70,4,-0.075000,1.35,1,0.350000,941.220,697.2,0.350000,80127.270,99180.0,-0.192103


In [68]:
print(f" Forecasting for Quarter: {next_quarter_str}")
display(territory_forecast)

 Forecasting for Quarter: 2015-Q3


,territory_id,forecast_quarter,forecast_deals,forecast_wins,forecast_win_rate,forecast_amount,forecast_sales
0,1,2015-Q3,7.70,6.45,0.821943,181151.850,154555.890
1,2,2015-Q3,5.40,5.35,0.891577,128082.600,119520.700
2,3,2015-Q3,51.35,35.50,0.700396,1354761.300,1066682.140
3,4,2015-Q3,4.85,2.70,0.576667,95054.160,12879.000
4,5,2015-Q3,9.55,8.30,0.805076,222385.525,187381.375
5,6,2015-Q3,32.15,18.85,0.581924,831959.940,509127.800
6,7,2015-Q3,3.80,3.30,0.857500,95294.580,69459.150
7,8,2015-Q3,4.00,2.25,0.602083,203202.075,165930.025
8,9,2015-Q3,21.80,13.55,0.621751,541814.050,376826.220
9,10,2015-Q3,3.70,1.35,0.433333,80127.270,941.220


In [70]:
# ------------------------------------------------
# Rep-Level Performance by Territory and Quarter
# ------------------------------------------------

rep_territory_quarter_summary = (
    deals_clean
    .groupby(['account_owned_by', 'territory_id', 'year_quarter'])
    .agg(
        deals=('opportunity_id', 'count'),
        wins=('won', 'sum'),
        win_rate=('won', 'mean'),
        total_amount=('amount', 'sum'),
        total_weekly_sales=('weekly_sales', 'sum')
    )
    .reset_index()
    .sort_values(by=['account_owned_by', 'territory_id', 'year_quarter'])
)

# Preview
display(rep_territory_quarter_summary.head(10))

,account_owned_by,territory_id,year_quarter,deals,wins,win_rate,total_amount,total_weekly_sales
0,david ward,7,2012-Q4,1,0,0.000000,1764.0,0.0
1,david ward,7,2013-Q4,3,2,0.666667,5913.0,5049.0
2,david ward,7,2014-Q4,1,0,0.000000,213.6,0.0
3,david ward,7,2015-Q1,2,0,0.000000,2592.0,0.0
4,david ward,8,2012-Q4,1,1,1.000000,10080.0,10080.0
5,david ward,8,2013-Q4,2,1,0.500000,5424.0,1896.0
6,david ward,8,2014-Q2,1,1,1.000000,2362.5,2362.5
7,david ward,8,2014-Q4,3,0,0.000000,18574.0,0.0
8,david ward,9,2011-Q2,1,1,1.000000,6776.0,6776.0
9,david ward,9,2012-Q2,1,1,1.000000,101600.0,101600.0


In [72]:
# ---------------------------------------------
# Get Most Recent Quarter (Q2) Actuals at Territory Level
# ---------------------------------------------
recent_actuals = territory_performance_validation[[
    'territory_id', 'recent_quarter',
    'recent_deals', 'recent_wins', 'recent_win_rate',
    'recent_amount', 'recent_weekly_sales'
]].rename(columns={
    'recent_quarter': 'previous_quarter',
    'recent_deals': 'prev_deals',
    'recent_wins': 'prev_wins',
    'recent_win_rate': 'prev_win_rate',
    'recent_amount': 'prev_amount',
    'recent_weekly_sales': 'prev_weekly_sales'
})

# ---------------------------------------------
# Merge Territory Forecast with Actuals
# ---------------------------------------------
comparison_df = pd.merge(
    territory_forecast,
    recent_actuals,
    on='territory_id',
    how='left'
)

# ---------------------------------------------
# Calculate % Change from Q2 to Q3
# ---------------------------------------------
comparison_df['deals_pct_change'] = (comparison_df['forecast_deals'] - comparison_df['prev_deals']) / comparison_df['prev_deals']
comparison_df['wins_pct_change'] = (comparison_df['forecast_wins'] - comparison_df['prev_wins']) / comparison_df['prev_wins']
comparison_df['sales_pct_change'] = (comparison_df['forecast_sales'] - comparison_df['prev_weekly_sales']) / comparison_df['prev_weekly_sales']
comparison_df['amount_pct_change'] = (comparison_df['forecast_amount'] - comparison_df['prev_amount']) / comparison_df['prev_amount']

# ---------------------------------------------
# Get Rep-Level Contribution in Previous Quarter (Q2)
# ---------------------------------------------
q2_rep_contributions = (
    rep_territory_quarter_summary[
        rep_territory_quarter_summary['year_quarter'] == '2015-Q2'
    ][[
        'account_owned_by', 'territory_id', 'total_weekly_sales'
    ]]
    .groupby(['territory_id', 'account_owned_by'], as_index=False)
    .agg(rep_sales_contribution_q2=('total_weekly_sales', 'sum'))
)

# ---------------------------------------------
# Merge Rep Contributions Into Comparison
# ---------------------------------------------
final_output = pd.merge(
    comparison_df,
    q2_rep_contributions,
    on='territory_id',
    how='left'
)

# ---------------------------------------------
# Sort for Clarity
# ---------------------------------------------
final_output = final_output.sort_values(by=['account_owned_by', 'territory_id'])


# Preview Final Output
display(final_output.head(20))

,territory_id,forecast_quarter,forecast_deals,forecast_wins,forecast_win_rate,forecast_amount,forecast_sales,previous_quarter,prev_deals,prev_wins,prev_win_rate,prev_amount,prev_weekly_sales,deals_pct_change,wins_pct_change,sales_pct_change,amount_pct_change,account_owned_by,rep_sales_contribution_q2
29,9,2015-Q3,21.80,13.55,0.621751,541814.050,376826.220,2015-Q2,20,12,0.600000,478244.0,420431.0,0.090000,0.129167,-0.103714,0.132924,david ward,176982.0
0,1,2015-Q3,7.70,6.45,0.821943,181151.850,154555.890,2015-Q2,7,6,0.857143,152113.2,151366.2,0.100000,0.075000,0.021073,0.190902,deepak ramirez,71262.0
4,2,2015-Q3,5.40,5.35,0.891577,128082.600,119520.700,2015-Q2,4,4,1.000000,94876.0,94876.0,0.350000,0.337500,0.259757,0.350000,deepak ramirez,0.0
8,3,2015-Q3,51.35,35.50,0.700396,1354761.300,1066682.140,2015-Q2,46,34,0.739130,1345072.6,1211646.4,0.116304,0.044118,-0.119642,0.007203,deepak ramirez,91344.0
1,1,2015-Q3,7.70,6.45,0.821943,181151.850,154555.890,2015-Q2,7,6,0.857143,152113.2,151366.2,0.100000,0.075000,0.021073,0.190902,dennis watson,16020.0
5,2,2015-Q3,5.40,5.35,0.891577,128082.600,119520.700,2015-Q2,4,4,1.000000,94876.0,94876.0,0.350000,0.337500,0.259757,0.350000,dennis watson,24536.0
9,3,2015-Q3,51.35,35.50,0.700396,1354761.300,1066682.140,2015-Q2,46,34,0.739130,1345072.6,1211646.4,0.116304,0.044118,-0.119642,0.007203,dennis watson,100898.4
39,13,2015-Q3,20.65,17.80,0.859797,440689.530,424252.470,2015-Q2,25,22,0.880000,492629.4,487278.6,-0.174000,-0.190909,-0.129343,-0.105434,elissa sanders,80502.0
43,14,2015-Q3,12.50,10.40,0.821226,198007.200,165971.450,2015-Q2,14,12,0.857143,146672.0,133271.0,-0.107143,-0.133333,0.245368,0.350000,elissa sanders,47740.0
47,15,2015-Q3,70.20,46.65,0.646662,1935293.150,1064722.940,2015-Q2,78,51,0.653846,2245788.0,1166824.0,-0.100000,-0.085294,-0.087503,-0.138257,elissa sanders,150316.8


In [74]:
rep_check = (
    q2_rep_contributions
    .groupby('territory_id')['rep_sales_contribution_q2']
    .sum()
    .reset_index()
    .merge(recent_actuals[['territory_id', 'prev_weekly_sales']], on='territory_id')
)
rep_check['contribution_match'] = np.isclose(rep_check['rep_sales_contribution_q2'], rep_check['prev_weekly_sales'])
display(rep_check)

,territory_id,rep_sales_contribution_q2,prev_weekly_sales,contribution_match
0,1,151366.2,151366.2,True
1,2,94876.0,94876.0,True
2,3,1211646.4,1211646.4,True
3,4,9540.0,9540.0,True
4,5,218440.0,218440.0,True
5,6,410674.6,410674.6,True
6,7,90392.4,90392.4,True
7,8,182676.0,182676.0,True
8,9,420431.0,420431.0,True
9,10,697.2,697.2,True


---

## Modeling

In [77]:
# ---------------------------------------------
# Get Most Recent Quarter (Q2) Actuals at Territory Level
# ---------------------------------------------
recent_actuals = territory_performance_validation[[
    'territory_id', 'recent_quarter',
    'recent_deals', 'recent_wins', 'recent_win_rate',
    'recent_amount', 'recent_weekly_sales'
]].rename(columns={
    'recent_quarter': 'previous_quarter',
    'recent_deals': 'prev_deals',
    'recent_wins': 'prev_wins',
    'recent_win_rate': 'prev_win_rate',
    'recent_amount': 'prev_amount',
    'recent_weekly_sales': 'prev_weekly_sales'
})

# ---------------------------------------------
# Merge Territory Forecast with Actuals
# ---------------------------------------------
comparison_df = pd.merge(
    territory_forecast,
    recent_actuals,
    on='territory_id',
    how='left'
)

# ---------------------------------------------
# Calculate % Change from Q2 to Q3
# ---------------------------------------------
comparison_df['deals_pct_change'] = (comparison_df['forecast_deals'] - comparison_df['prev_deals']) / comparison_df['prev_deals']
comparison_df['wins_pct_change'] = (comparison_df['forecast_wins'] - comparison_df['prev_wins']) / comparison_df['prev_wins']
comparison_df['sales_pct_change'] = (comparison_df['forecast_sales'] - comparison_df['prev_weekly_sales']) / comparison_df['prev_weekly_sales']
comparison_df['amount_pct_change'] = (comparison_df['forecast_amount'] - comparison_df['prev_amount']) / comparison_df['prev_amount']

# ---------------------------------------------
# Get Rep-Level Sales Contribution (Q2)
# ---------------------------------------------
q2_rep_contributions = (
    rep_territory_quarter_summary[
        rep_territory_quarter_summary['year_quarter'] == '2015-Q2'
    ][['account_owned_by', 'territory_id', 'total_weekly_sales']]
    .groupby(['territory_id', 'account_owned_by'], as_index=False)
    .agg(rep_sales_q2=('total_weekly_sales', 'sum'))
)

# ---------------------------------------------
# Merge Rep Contributions with Territory Comparison
# ---------------------------------------------
rep_contribution_with_forecast = pd.merge(
    q2_rep_contributions,
    comparison_df[['territory_id', 'forecast_quarter', 'forecast_sales', 'prev_weekly_sales']],
    on='territory_id',
    how='left'
)

# ---------------------------------------------
# Calculate % Contribution + Rep-Level Q3 Forecast
# ---------------------------------------------
rep_contribution_with_forecast['rep_sales_contribution_ratio'] = (
    rep_contribution_with_forecast['rep_sales_q2'] / rep_contribution_with_forecast['prev_weekly_sales']
)

rep_contribution_with_forecast['rep_expected_sales_q3'] = (
    rep_contribution_with_forecast['forecast_sales'] * rep_contribution_with_forecast['rep_sales_contribution_ratio']
)

# ---------------------------------------------
# Sort & Preview
# ---------------------------------------------
rep_contribution_with_forecast = rep_contribution_with_forecast.sort_values(by=['account_owned_by', 'territory_id'])

display(rep_contribution_with_forecast.head(10))

,territory_id,account_owned_by,rep_sales_q2,forecast_quarter,forecast_sales,prev_weekly_sales,rep_sales_contribution_ratio,rep_expected_sales_q3
29,9,david ward,176982.0,2015-Q3,376826.22,420431.0,0.420954,158626.404970
0,1,deepak ramirez,71262.0,2015-Q3,154555.89,151366.2,0.470792,72763.680618
4,2,deepak ramirez,0.0,2015-Q3,119520.70,94876.0,0.000000,0.000000
8,3,deepak ramirez,91344.0,2015-Q3,1066682.14,1211646.4,0.075388,80415.386367
1,1,dennis watson,16020.0,2015-Q3,154555.89,151366.2,0.105836,16357.584175
5,2,dennis watson,24536.0,2015-Q3,119520.70,94876.0,0.258611,30909.396425
9,3,dennis watson,100898.4,2015-Q3,1066682.14,1211646.4,0.083274,88826.675204
39,13,elissa sanders,80502.0,2015-Q3,424252.47,487278.6,0.165207,70089.620886
43,14,elissa sanders,47740.0,2015-Q3,165971.45,133271.0,0.358217,59453.872358
47,15,elissa sanders,150316.8,2015-Q3,1064722.94,1166824.0,0.128826,137163.569851


In [79]:
rep_check = (
    rep_contribution_with_forecast
    .groupby('territory_id')['rep_sales_q2']
    .sum()
    .reset_index()
    .merge(recent_actuals[['territory_id', 'prev_weekly_sales']], on='territory_id')
)
rep_check['contribution_match'] = np.isclose(rep_check['rep_sales_q2'], rep_check['prev_weekly_sales'])
display(rep_check)

,territory_id,rep_sales_q2,prev_weekly_sales,contribution_match
0,1,151366.2,151366.2,True
1,2,94876.0,94876.0,True
2,3,1211646.4,1211646.4,True
3,4,9540.0,9540.0,True
4,5,218440.0,218440.0,True
5,6,410674.6,410674.6,True
6,7,90392.4,90392.4,True
7,8,182676.0,182676.0,True
8,9,420431.0,420431.0,True
9,10,697.2,697.2,True


In [81]:
# ---------------------------------------------
# Add quarter_index to rep_territory_quarter_summary
# ---------------------------------------------
rep_territory_quarter_summary = rep_territory_quarter_summary.copy()

rep_territory_quarter_summary['year'] = rep_territory_quarter_summary['year_quarter'].str[:4].astype(int)
rep_territory_quarter_summary['q'] = rep_territory_quarter_summary['year_quarter'].str[-1].astype(int)
rep_territory_quarter_summary['quarter_index'] = (rep_territory_quarter_summary['year'] * 4) + rep_territory_quarter_summary['q']

# ---------------------------------------------
# Filter clean rep-quarter records (exclude Q3 and 0 sales)
# ---------------------------------------------
rep_valid = rep_territory_quarter_summary[
    (rep_territory_quarter_summary['year_quarter'] != '2015-Q3') &
    (rep_territory_quarter_summary['total_weekly_sales'] > 0)
].copy()

# Rank by quarter_index (descending = latest first)
rep_valid['rank'] = (
    rep_valid
    .groupby(['account_owned_by', 'territory_id'])['quarter_index']
    .rank(method='first', ascending=False)
)

# Separate recent (top 2) and historical (rest)
recent_sales = rep_valid[rep_valid['rank'] <= 2]
historical_sales = rep_valid[rep_valid['rank'] > 2]

# Compute average sales
recent_avg = recent_sales.groupby(['account_owned_by', 'territory_id'])['total_weekly_sales'].mean().reset_index()
recent_avg.rename(columns={'total_weekly_sales': 'avg_recent_sales'}, inplace=True)

historical_avg = historical_sales.groupby(['account_owned_by', 'territory_id'])['total_weekly_sales'].mean().reset_index()
historical_avg.rename(columns={'total_weekly_sales': 'avg_historical_sales'}, inplace=True)

# Merge and calculate weighted avg
rep_weighted_avg = pd.merge(recent_avg, historical_avg, on=['account_owned_by', 'territory_id'], how='outer')

rep_weighted_avg['avg_recent_sales'] = rep_weighted_avg['avg_recent_sales'].fillna(0)
rep_weighted_avg['avg_historical_sales'] = rep_weighted_avg['avg_historical_sales'].fillna(0)

rep_weighted_avg['rep_weighted_avg_sales'] = (
    0.7 * rep_weighted_avg['avg_recent_sales'] + 
    0.3 * rep_weighted_avg['avg_historical_sales']
)

# Preview the weighted sales result
display(rep_weighted_avg.head(10))

,account_owned_by,territory_id,avg_recent_sales,avg_historical_sales,rep_weighted_avg_sales
0,david ward,7,5049.00,0.000000,3534.300000
1,david ward,8,2129.25,10080.000000,4514.475000
2,david ward,9,92347.50,70828.900000,85891.920000
3,deepak ramirez,1,57550.50,98416.800000,69810.390000
4,deepak ramirez,2,27038.00,51567.000000,34396.700000
5,deepak ramirez,3,91594.80,111011.750000,97419.885000
6,dennis watson,1,47494.80,10711.920000,36459.936000
7,dennis watson,2,176325.00,18446.285714,128961.385714
8,dennis watson,3,133466.40,192991.766667,151324.010000
9,elissa sanders,13,89824.80,32373.600000,72589.440000


In [83]:
# ---------------------------------------------
# Get Most Recent Quarter (Q2) Actuals at Territory Level
# ---------------------------------------------
recent_actuals = territory_performance_validation[[
    'territory_id', 'recent_quarter',
    'recent_deals', 'recent_wins', 'recent_win_rate',
    'recent_amount', 'recent_weekly_sales'
]].rename(columns={
    'recent_quarter': 'previous_quarter',
    'recent_deals': 'prev_deals',
    'recent_wins': 'prev_wins',
    'recent_win_rate': 'prev_win_rate',
    'recent_amount': 'prev_amount',
    'recent_weekly_sales': 'prev_weekly_sales'
})

# ---------------------------------------------
# Merge Territory Forecast with Actuals
# ---------------------------------------------
comparison_df = pd.merge(
    territory_forecast,
    recent_actuals,
    on='territory_id',
    how='left'
)

# ---------------------------------------------
# Calculate % Change from Q2 to Q3
# ---------------------------------------------
comparison_df['deals_pct_change'] = (comparison_df['forecast_deals'] - comparison_df['prev_deals']) / comparison_df['prev_deals']
comparison_df['wins_pct_change'] = (comparison_df['forecast_wins'] - comparison_df['prev_wins']) / comparison_df['prev_wins']
comparison_df['sales_pct_change'] = (comparison_df['forecast_sales'] - comparison_df['prev_weekly_sales']) / comparison_df['prev_weekly_sales']
comparison_df['amount_pct_change'] = (comparison_df['forecast_amount'] - comparison_df['prev_amount']) / comparison_df['prev_amount']

# ---------------------------------------------
# Get Rep-Level Sales Contribution (Q2)
# ---------------------------------------------
q2_rep_contributions = (
    rep_territory_quarter_summary[
        rep_territory_quarter_summary['year_quarter'] == '2015-Q2'
    ][['account_owned_by', 'territory_id', 'total_weekly_sales']]
    .groupby(['territory_id', 'account_owned_by'], as_index=False)
    .agg(rep_sales_q2=('total_weekly_sales', 'sum'))
)


# ---------------------------------------------
# Merge Rep Contributions with Territory Comparison
# ---------------------------------------------
rep_contribution_with_forecast = pd.merge(
    q2_rep_contributions,
    comparison_df[['territory_id', 'forecast_quarter', 'forecast_sales', 'prev_weekly_sales']],
    on='territory_id',
    how='left'
)

# ---------------------------------------------
# Calculate % Contribution + Rep-Level Q3 Forecast
# ---------------------------------------------
rep_contribution_with_forecast['rep_sales_contribution_ratio'] = (
    rep_contribution_with_forecast['rep_sales_q2'] / rep_contribution_with_forecast['prev_weekly_sales']
)

rep_contribution_with_forecast['rep_expected_sales_q3'] = (
    rep_contribution_with_forecast['forecast_sales'] * rep_contribution_with_forecast['rep_sales_contribution_ratio']
)

# ---------------------------------------------
# Merge Rep Weighted Avg into Contribution + Forecast
# ---------------------------------------------
rep_contribution_with_forecast = pd.merge(
    rep_contribution_with_forecast,
    rep_weighted_avg[['account_owned_by', 'territory_id', 'rep_weighted_avg_sales']],
    on=['account_owned_by', 'territory_id'],
    how='left'
)

# ---------------------------------------------
# Sort & Preview
# ---------------------------------------------
rep_contribution_with_forecast = rep_contribution_with_forecast.sort_values(by=['account_owned_by', 'territory_id'])

display(rep_contribution_with_forecast.head(10))




,territory_id,account_owned_by,rep_sales_q2,forecast_quarter,forecast_sales,prev_weekly_sales,rep_sales_contribution_ratio,rep_expected_sales_q3,rep_weighted_avg_sales
29,9,david ward,176982.0,2015-Q3,376826.22,420431.0,0.420954,158626.404970,85891.920000
0,1,deepak ramirez,71262.0,2015-Q3,154555.89,151366.2,0.470792,72763.680618,69810.390000
4,2,deepak ramirez,0.0,2015-Q3,119520.70,94876.0,0.000000,0.000000,34396.700000
8,3,deepak ramirez,91344.0,2015-Q3,1066682.14,1211646.4,0.075388,80415.386367,97419.885000
1,1,dennis watson,16020.0,2015-Q3,154555.89,151366.2,0.105836,16357.584175,36459.936000
5,2,dennis watson,24536.0,2015-Q3,119520.70,94876.0,0.258611,30909.396425,128961.385714
9,3,dennis watson,100898.4,2015-Q3,1066682.14,1211646.4,0.083274,88826.675204,151324.010000
39,13,elissa sanders,80502.0,2015-Q3,424252.47,487278.6,0.165207,70089.620886,72589.440000
43,14,elissa sanders,47740.0,2015-Q3,165971.45,133271.0,0.358217,59453.872358,89281.162500
47,15,elissa sanders,150316.8,2015-Q3,1064722.94,1166824.0,0.128826,137163.569851,176938.578182


---

## Bayesian Model



In [86]:
# Fallback std deviation if none exists
fallback_std_ratio = 0.25

# Storage for results
bayes_results = []

# ---------------------------------------------
# Filter to single rep–territory row
# ---------------------------------------------
rep_row = rep_contribution_with_forecast[
    (rep_contribution_with_forecast['account_owned_by'] == 'deepak ramirez') &
    (rep_contribution_with_forecast['territory_id'] == 1)
].copy()

if not rep_row.empty:
    row = rep_row.reset_index(drop=True).iloc[0]
    mu = row['rep_weighted_avg_sales']
    expected_sales_q3 = row['rep_expected_sales_q3']
    std = row.get('std_weekly_sales', np.nan)

    # Handle missing or zero values early
    if pd.isna(mu) or pd.isna(expected_sales_q3) or expected_sales_q3 == 0:
        result = {
            'posterior_mean_sales': np.nan,
            'prob_meet_or_exceed': np.nan,
            'credible_interval_lower': np.nan,
            'credible_interval_upper': np.nan,
            'sales_pressure_ratio': np.nan,
            'risk_flag': 'insufficient data'
        }
    else:
        if pd.isna(std) or std == 0:
            std = mu * fallback_std_ratio

        with pm.Model():
            sales = pm.Normal("sales", mu=mu, sigma=std)
            trace = pm.sample(2000, tune=1000, chains=2, progressbar=False)
            sales_samples = trace.posterior['sales'].stack(draws=("chain", "draw")).values

        posterior_mean = np.mean(sales_samples)
        prob_hit = np.mean(sales_samples >= expected_sales_q3)
        lower, upper = np.percentile(sales_samples, [3, 97])
        pressure_ratio = expected_sales_q3 / mu

        # Risk category
        if prob_hit < 0.33:
            risk_flag = 'high risk'
        elif prob_hit < 0.66:
            risk_flag = 'moderate risk'
        else:
            risk_flag = 'low risk'

        result = {
            'posterior_mean_sales': posterior_mean,
            'prob_meet_or_exceed': prob_hit,
            'credible_interval_lower': lower,
            'credible_interval_upper': upper,
            'sales_pressure_ratio': pressure_ratio,
            'risk_flag': risk_flag
        }

    # Append result
    bayes_results.append(result)

    # Combine with original row
    single_bayes_output = pd.concat([rep_row.reset_index(drop=True), pd.DataFrame(bayes_results)], axis=1)

    # Preview
    display(single_bayes_output)
else:
    print("No matching row found for that rep and territory.")

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sales]
Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 8 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


,territory_id,account_owned_by,rep_sales_q2,forecast_quarter,forecast_sales,prev_weekly_sales,rep_sales_contribution_ratio,rep_expected_sales_q3,rep_weighted_avg_sales,posterior_mean_sales,prob_meet_or_exceed,credible_interval_lower,credible_interval_upper,sales_pressure_ratio,risk_flag
0,1,deepak ramirez,71262.0,2015-Q3,154555.89,151366.2,0.470792,72763.680618,69810.39,69549.141505,0.4295,36559.636027,102727.205416,1.042304,moderate risk
